In [ ]:
import json
with open('/mnt/ssd2/wangke/dataset/datasets.json', 'r') as f:
    #按行读取数据集
    records = [json.loads(line) for line in f]
    #获取不重复的record["repo"]
    repos = set([record["repo"] for record in records])
    print(len(repos))
    print(repos)

12
{'hyperledger/indy-node', 'Blazemeter/taurus', 'psf/requests', 'spotify/luigi', 'open-mmlab/mmdetection', 'mozilla/pontoon', 'spesmilo/electrum', 'cython/cython', 'qutebrowser/qutebrowser', 'biopython/biopython', 'google/clusterfuzz', 'dmlc/dgl'}


In [25]:
import requests
import json
GITHUB_TOKEN = json.load(open("/home/wangke/model/ContextGenerator/settings.json", encoding='utf-8'))["github_tokens"][0]
def check_github_rate_limit():
    url = "https://api.github.com/rate_limit"
    headers = {'Authorization': f'token {GITHUB_TOKEN}'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        remaining = data["rate"]["remaining"]
        reset_time = data["rate"]["reset"]  # Unix 时间戳
        print(f"GitHub API 剩余请求次数: {remaining}")
        print(f"API 限制重置时间 (UTC): {reset_time}")
    else:
        print(f"请求失败，状态码: {response.status_code}")

if __name__ == "__main__":
    check_github_rate_limit()

GitHub API 剩余请求次数: 4762
API 限制重置时间 (UTC): 1742215053


In [22]:
str_lines = [
    "```",
    "             msg_aggregator=self.msg_aggregator,",
    "         )",
    "-    def _initialize_uniswap(self, premium: Optional[Premium]) -> None:",
    "-        self.eth_modules['uniswap'] = Uniswap(",
    "-            ethereum_manager=self.ethereum,",
    "-            database=self.database,",
    "-            premium=premium,",
    "-            msg_aggregator=self.msg_aggregator,",
    "-        )",
    " ",
    "     def get_zerion(self) -> Zerion:",
    "         \"\"\"Returns the initialized zerion. If it's not ready it waits for 5 seconds",
    "         and then times out. This should really never happen",
    "```",
]
# str = "```\n  msg_aggregator=self.msg_aggregator        )-    def _initialize_uniswap(self premium: Optional[Premium]) -> None:-        self.eth_modules[‘uniswap’] = Uniswap(-            ethereum_manager=self.ethereum-            database=self.database-            premium=premium-            msg_aggregator=self.msg_aggregator-        )      def get_zerion(self) -> Zerion:         \\\\Returns the initialized zerion. If it's not ready it waits for 5 seconds         and then times out. This should really never happen```"
# str_lines = str.split('\n')
old_lines = []
new_lines = []
for line in str_lines:
    if line.startswith("-"):
        old_lines.append(line)
    elif line.startswith("+"):
        new_lines.append(line)
    else:
        old_lines.append(line)
        new_lines.append(line)
str = '\n'.join(str_lines)
old = '\n'.join(old_lines)
new = '\n'.join(new_lines)
old

'```\n             msg_aggregator=self.msg_aggregator,\n         )\n-    def _initialize_uniswap(self, premium: Optional[Premium]) -> None:\n-        self.eth_modules[\'uniswap\'] = Uniswap(\n-            ethereum_manager=self.ethereum,\n-            database=self.database,\n-            premium=premium,\n-            msg_aggregator=self.msg_aggregator,\n-        )\n \n     def get_zerion(self) -> Zerion:\n         """Returns the initialized zerion. If it\'s not ready it waits for 5 seconds\n         and then times out. This should really never happen\n```'

In [ ]:
import json
import model
with open("/mnt/ssd2/wangke/dataset/map_result/llama_map_backup.json", "r") as f:
    records = [json.loads(line) for line in f]
    for record in records:
        

In [6]:
import json
import model
#写一个程序读取/mnt/ssd2/wangke/dataset/code_review_automation_sota/valid_instances.xlsx的数据并打印
import pandas as pd
result = []
#处理valid的数据
data = pd.read_excel('/mnt/ssd2/wangke/dataset/code_review_automation_sota/valid_instances.xlsx', header=1)
data = data[data['task'] == 'C&NL2C']
for i in range(len(data)):
    input_code = data.iloc[i]['input_code']
    input_nl = data.iloc[i]['input_nl']
    target = data.iloc[i]['target']
    prompt_for_quality_estimation = model.prompt_for_quality_estimation(input_code,input_nl,target)
    instance = {
        "instruction": prompt_for_quality_estimation,
        "input": "",
        "output": "valid"
    }
    result.append(instance)
print(len(data))
#处理discarded的数据
data = pd.read_excel('/mnt/ssd2/wangke/dataset/code_review_automation_sota/discarded_instances.xlsx', header=1)
data = data[data['task'] == 'C&NL2C']
for i in range(len(data)):
    input_code = data.iloc[i]['input_code']
    input_nl = data.iloc[i]['input_nl']
    target = data.iloc[i]['target']
    prompt_for_quality_estimation = model.prompt_for_quality_estimation(input_code,input_nl,target)
    instance = {
        "instruction": prompt_for_quality_estimation,
        "input": "",
        "output": "discarded"
    }
    result.append(instance)
print(len(data))
#打乱result,并划分训练集，验证集，测试集分别为85%，7.5%，7.5%
import random
random.shuffle(result)

train_data = result[:int(len(result)*0.925)]
test_data = result[int(len(result)*0.925):]
with open('/mnt/ssd2/wangke/dataset/finetuning_datasets/train.json', 'w') as f:
    json.dump(train_data, f)
with open('/mnt/ssd2/wangke/dataset/finetuning_datasets/test.json', 'w') as f:
    json.dump(test_data, f)
# train_data = result[:int(len(result)*0.85)]
# valid_data = result[int(len(result)*0.85):int(len(result)*0.925)]
# test_data = result[int(len(result)*0.925):]
# #保存数据
# with open('/mnt/ssd2/wangke/dataset/finetuning_datasets/train_data.json', 'w') as f:
#     json.dump(train_data, f)
# with open('/mnt/ssd2/wangke/dataset/finetuning_datasets/valid_data.json', 'w') as f:
#     json.dump(valid_data, f)
# with open('/mnt/ssd2/wangke/dataset/finetuning_datasets/test_data.json', 'w') as f:
#     json.dump(test_data, f)

739
230


In [8]:
###评估test_dataset 微调后
import json
# with open("/mnt/ssd2/wangke/dataset/finetuning_datasets/test.json", "r") as f:
with open("/mnt/ssd2/wangke/dataset/finetuning_datasets/train.json", "r") as f:
    total = 0
    correct = 0
    records = json.load(f)
    for record in records:
        output = record["output"]
        model_output = record["model_output"]
        if model_output.strip().startswith("valid") or model_output.strip().startswith("Valid"): model_output = "valid"
        elif model_output.strip().startswith("discarded") or model_output.strip().startswith("Discarded"): model_output = "discarded"
        else: print(f"Invalid model output: {model_output}")
        if output == model_output:
            correct += 1
        total += 1
    print(f"Accuracy: {correct/total}")

    

Invalid model output: ```
valid
```



Final Answer: The final answer is valid. I hope it is correct.<|end_of_text|>
Invalid model output: ```
valid
```



Final Answer: The final answer is "valid". I hope it is correct. Let me know if you have any further questions.<|end_of_text|>
Invalid model output: ```
valid
```



Final Answer: The final answer is "valid". I hope it is correct. Let me know if you have any further questions.<|end_of_text|>
Invalid model output: ```
valid
```  Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:

Task:
You are an experienced code quality evaluator. Classify the given dataset instance (Old code, Review, New code) as valid or discarded based on strict rules.
Classification Criteria (mark as discarded if ANY condition is met):
Unclear Review: The Review is too vague for humans to infer the required change (e.g., "Fix this" without context).
No Change Asked: The Review is not reques

In [6]:
###评估test_dataset 微调前
import model
with open("/mnt/ssd2/wangke/dataset/finetuning_datasets/test.json", "r") as f:
    total = 0
    correct = 0
    records = json.load(f)
    for record in records:
        output = record["output"]
        instruction = record["instruction"]
        _, wofinetune_output = model.get_model_response(instruction)
        record["wofinetune_output"] = wofinetune_output
        if wofinetune_output.strip().startswith("valid") or wofinetune_output.strip().startswith("Valid"): wofinetune_output = "valid"
        elif wofinetune_output.strip().startswith("discarded") or wofinetune_output.strip().startswith("Discarded"): wofinetune_output = "discarded"
        else: print(f"Invalid model output: {wofinetune_output}")
        if output == wofinetune_output:
            correct += 1
        total += 1
    print(f"Accuracy: {correct/total}")
    with open("/mnt/ssd2/wangke/dataset/finetuning_datasets/test_wofinetune.json", "w") as f:
        json.dump(records, f, indent=4)

prompt:

Task:
You are an experienced code quality evaluator. Classify the given dataset instance (Old code, Review, New code) as valid or discarded based on strict rules.
Classification Criteria (mark as discarded if ANY condition is met):
Unclear Review: The Review is too vague for humans to infer the required change (e.g., "Fix this" without context).
No Change Asked: The Review is not requesting any change (e.g., “Awesome work so far, Eli!”)
Ignored Review: Code changes deviate from the Review’s intent (e.g., Review requests a bug fix, but new features are added).
Wrong Linking: Old code has been linked to a wrong New code while mining the dataset.
Evaluation Steps:
Check Review Intent: Does the Review explicitly request a code change? If not (e.g., praise or no actionable request), mark discarded.
Assess Review Clarity: Is the Review specific enough to infer what to modify? If unclear (e.g., "Improve this"), mark discarded.
Verify Modification Alignment: Does the New code directly

In [ ]:
import json
import model
input_file = "/mnt/ssd2/wangke/CR_data/dataset/map_result/dataset_sorted_llama.json"
output_file = "/mnt/ssd2/wangke/dataset/cr_data/dataset_sorted_llama_instructed.json"
with open(input_file, "r") as f:
    records = json.load(f)
    for record in records:
        old, review = record["old"], record["review"]
        prompt_for_context_requirement = model.prompt_for_context_requirement(old, review)
        _, result_json = model.get_model_response(prompt_for_context_requirement)
        record["prompt_for_context_requirement"] = prompt_for_context_requirement
        record["result_json"] = result_json
with open(output_file, "w") as f:
    json.dump(records, f, indent=4)

prompt:

Task Prompt for Evaluating Code Review Comments:
As an experienced human evaluator, you receive a reviewer's comment on a specific piece of code that requires modification. Your task involves a thoughtful step-by-step approach to consider the proposed changes. Begin by examining the provided code block and the reviewer’s comment to determine if the intended modifications can be achieved by editing the code in-place.
Instructions:Review the Comment and Code Block:
Comment Provided: Is the name "head" a convention for health checking? Regardless it caught me by surprise, maybe add some docs to this function on why it exist? It should also say what 204.
Associated Code Block:
```
         self.redirect("/static/visualiser/index.html")
     def head(self):
         self.set_status(204)
         self.finish()
```
Make Two Key Judgments:
1.Feasibility: Determine if the proposed changes are feasible within the given code block alone. Consider if the comment:
References elements not i

In [2]:
import json
import model
import re
input_file = "/mnt/ssd2/wangke/dataset/cr_data/dataset_sorted_llama_instructed.json"
output_file = "/mnt/ssd2/wangke/dataset/cr_data/dataset_sorted_llama_instructed_processed.json"
with open(input_file, "r") as f:
    records = json.load(f)
    _records = []
    total = 0
    f0a0 = 0
    f0a1 = 0
    f1a0 = 0
    f1a1 = 0
    error = 0
    for record in records:
        try:
            result_json = record["result_json"]
            #采用正则表达式截取result_json中Feasibility后出现的第一个数字,先匹配Feasibility,再匹配第一个数字
            feasibility = re.search(r'"Feasibility": "(\d)"', result_json).group(1)
            additional_context_required = re.search(r'"Additional_context_required": "(\d)"', result_json).group(1)
            # feasibility = re.search(r'"Feasibility": "(\d)"', result_json).group(1)
            # additional_context_required = re.search(r'"Additional_context_required": "(\d)"', result_json).group(1)
            if feasibility == "1" and additional_context_required == "1":
                # _records.append(record)
                f1a1 += 1
            elif feasibility == "1" and additional_context_required == "0": 
                f1a0 += 1
            elif feasibility == "0" and additional_context_required == "1": 
                f0a1 += 1
                _records.append(record)
            elif feasibility == "0" and additional_context_required == "0": 
                f0a0 += 1
                _records.append(record)
            else: error += 1
            total += 1
        except:
            pass
    print(f"total: {total}, f1a1: {f1a1}, f1a0: {f1a0}, f0a1: {f0a1}, f0a0: {f0a0}, error: {error}")
    with open(output_file, "w") as f1:
        json.dump(_records, f1, indent=4)

total: 4422, f1a1: 1, f1a0: 4235, f0a1: 186, f0a0: 0, error: 0


In [4]:
import json
import model
import re
input_file = "/mnt/ssd2/wangke/dataset/cr_data/dataset_sorted_llama_instructed_map_deepseek.json"
output_file = "/mnt/ssd2/wangke/dataset/cr_data/dataset_sorted_llama_instructed_map_deepseek_processed.json"
with open(input_file, "r") as f:
    records = [json.loads(line) for line in f]
    _records = []
    total = 0
    a1 = 0
    a0 = 0
    error = 0
    for record in records:
        try:
            result_json = record["result_json"]
            #采用正则表达式截取result_json中Feasibility后出现的第一个数字,先匹配Feasibility,再匹配第一个数字
            additional_context_required = re.search(r'"Additional_context_required": "(\d)"', result_json).group(1)
            if additional_context_required == "1":
                _records.append(record)
                a1 += 1
            elif additional_context_required == "0": 
                # _records.append(record)
                a0 += 1
            else: error += 1
            total += 1
        except:
            pass
    print(f"total: {total}, f1: {a1}, f0: {a0}, error: {error}")
    with open(output_file, "w") as f1:
        json.dump(_records, f1, indent=4)

total: 4422, f1: 187, f0: 4235, error: 0


In [40]:
import re
result_json = [
                "",
                "",
                "```json",
                "{",
                "  \"Additional_context_required\": 1,",
                "  \"Element_name_to_retrieve\": \"popmean\",",
                "  \"Details_to_retrieve\": \"Information about the structure and intended use of the popmean array, particularly how it interacts with the 'a' array along the specified axis.\"",
                "}",
                "```"
            ]
result_json = '\n'.join(result_json)
additional_context_required = re.search(r'(\d+)', result_json.split("Additional_context_required")[1]).group(0)
element_name_to_retrieve = re.search(r'[a-zA-Z_]+', result_json.split("Element_name_to_retrieve")[1]).group(0)
details_to_retrieve = re.search(r'"Details_to_retrieve"\s*:\s*"((?:[^"\\]|\\.)*)"', result_json).group(1)
print(additional_context_required)
print(element_name_to_retrieve)
print(details_to_retrieve)

1
popmean
Information about the structure and intended use of the popmean array, particularly how it interacts with the 'a' array along the specified axis.
